# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 9 2022 12:03PM,239247,19,DEX0006972,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jun 9 2022 12:03PM,239247,19,DEX0006973,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 9 2022 12:03PM,239247,19,DEX0006974,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 9 2022 12:03PM,239247,19,DEX0006975,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jun 9 2022 12:03PM,239247,19,DEX0006976,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 9 2022 12:03PM,239247,19,DEX0006977,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jun 9 2022 12:03PM,239247,19,DEX0006978,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,Jun 9 2022 12:03PM,239247,19,DEX0006979,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,Jun 9 2022 11:49AM,239246,21,399290081,"NBTY Global, Inc.",Executing
9,Jun 9 2022 11:21AM,239244,10,SOTAH0000701,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,239241,Released,4
47,239243,Released,13
48,239244,Released,2
49,239246,Executing,1
50,239247,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239241,NaN,NaN,4.0
239243,NaN,NaN,13.0
239244,NaN,NaN,2.0
239246,NaN,1.0,NaN
239247,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239134,8.0,7.0,1.0
239144,0.0,1.0,0.0
239151,42.0,2.0,1.0
239154,4.0,8.0,3.0
239158,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239134,8,7,1
239144,0,1,0
239151,42,2,1
239154,4,8,3
239158,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239134,8,7,1
1,239144,0,1,0
2,239151,42,2,1
3,239154,4,8,3
4,239158,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239134,8,7,1
1,239144,,1,
2,239151,42,2,1
3,239154,4,8,3
4,239158,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 9 2022 12:03PM,239247,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
8,Jun 9 2022 11:49AM,239246,21,"NBTY Global, Inc."
9,Jun 9 2022 11:21AM,239244,10,"Nextsource Biotechnology, LLC"
11,Jun 9 2022 11:10AM,239243,20,"ACI Healthcare USA, Inc."
24,Jun 9 2022 11:01AM,239241,10,Bio-PRF
28,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc."
29,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC"
34,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc."
37,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC"
39,Jun 9 2022 10:06AM,239234,15,"Carwin Pharmaceutical Associates, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 9 2022 12:03PM,239247,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,8
1,Jun 9 2022 11:49AM,239246,21,"NBTY Global, Inc.",,1,
2,Jun 9 2022 11:21AM,239244,10,"Nextsource Biotechnology, LLC",,,2
3,Jun 9 2022 11:10AM,239243,20,"ACI Healthcare USA, Inc.",,,13
4,Jun 9 2022 11:01AM,239241,10,Bio-PRF,,,4
5,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc.",,,1
6,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC",,,5
7,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc.",,,3
8,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC",,1,1
9,Jun 9 2022 10:06AM,239234,15,"Carwin Pharmaceutical Associates, LLC",,7,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:03PM,239247,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8,,
1,Jun 9 2022 11:49AM,239246,21,"NBTY Global, Inc.",,1,
2,Jun 9 2022 11:21AM,239244,10,"Nextsource Biotechnology, LLC",2,,
3,Jun 9 2022 11:10AM,239243,20,"ACI Healthcare USA, Inc.",13,,
4,Jun 9 2022 11:01AM,239241,10,Bio-PRF,4,,
5,Jun 9 2022 10:17AM,239240,21,"NBTY Global, Inc.",1,,
6,Jun 9 2022 10:16AM,239239,15,"Virtus Pharmaceuticals, LLC",5,,
7,Jun 9 2022 10:15AM,239237,15,"Person & Covey, Inc.",3,,
8,Jun 9 2022 10:15AM,239235,15,"Laser Pharmaceuticals, LLC",1,1,
9,Jun 9 2022 10:06AM,239234,15,"Carwin Pharmaceutical Associates, LLC",1,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:03PM,239247,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8,,
1,Jun 9 2022 11:49AM,239246,21,"NBTY Global, Inc.",,1,
2,Jun 9 2022 11:21AM,239244,10,"Nextsource Biotechnology, LLC",2,,
3,Jun 9 2022 11:10AM,239243,20,"ACI Healthcare USA, Inc.",13,,
4,Jun 9 2022 11:01AM,239241,10,Bio-PRF,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:03PM,239247,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8.0,NaN,NaN
1,Jun 9 2022 11:49AM,239246,21,"NBTY Global, Inc.",NaN,1.0,NaN
2,Jun 9 2022 11:21AM,239244,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
3,Jun 9 2022 11:10AM,239243,20,"ACI Healthcare USA, Inc.",13.0,NaN,NaN
4,Jun 9 2022 11:01AM,239241,10,Bio-PRF,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 9 2022 12:03PM,239247,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,8.0,0.0,0.0
1,Jun 9 2022 11:49AM,239246,21,"NBTY Global, Inc.",0.0,1.0,0.0
2,Jun 9 2022 11:21AM,239244,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
3,Jun 9 2022 11:10AM,239243,20,"ACI Healthcare USA, Inc.",13.0,0.0,0.0
4,Jun 9 2022 11:01AM,239241,10,Bio-PRF,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1913684,31.0,14.0,5.0
15,1674641,23.0,17.0,0.0
18,717596,3.0,0.0,0.0
19,1913497,26.0,19.0,53.0
20,956820,17.0,7.0,8.0
21,1196141,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1913684,31.0,14.0,5.0
1,15,1674641,23.0,17.0,0.0
2,18,717596,3.0,0.0,0.0
3,19,1913497,26.0,19.0,53.0
4,20,956820,17.0,7.0,8.0
5,21,1196141,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,14.0,5.0
1,15,23.0,17.0,0.0
2,18,3.0,0.0,0.0
3,19,26.0,19.0,53.0
4,20,17.0,7.0,8.0
5,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,15,Released,23.0
2,18,Released,3.0
3,19,Released,26.0
4,20,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20,21
Status,,,,,,
Completed,5.0,0.0,0.0,53.0,8.0,0.0
Executing,14.0,17.0,0.0,19.0,7.0,1.0
Released,31.0,23.0,3.0,26.0,17.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20,21
0,Completed,5.0,0.0,0.0,53.0,8.0,0.0
1,Executing,14.0,17.0,0.0,19.0,7.0,1.0
2,Released,31.0,23.0,3.0,26.0,17.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20,21
0,Completed,5.0,0.0,0.0,53.0,8.0,0.0
1,Executing,14.0,17.0,0.0,19.0,7.0,1.0
2,Released,31.0,23.0,3.0,26.0,17.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()